In [115]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

print('[+] Налаштування завершено')

[+] Налаштування завершено


In [116]:
def load_data(file_path: str) -> pd.DataFrame:
    """
    Завантажує очищений датасет з вказаного шляху.

    :param file_path: Шлях до CSV файлу очищеного датасету.
    :return: DataFrame з завантаженими даними.
    """
    try:
        df = pd.read_csv(file_path)
        print("[+] Дані успішно завантажені.")
        return df
    except FileNotFoundError:
        print(f"[!] Файл за шляхом {file_path} не знайдено.")
        raise

In [117]:
DATA_FILE: str = '../../lab03/source/sample.csv'
DATAFRAME: pd.DataFrame = load_data(DATA_FILE)

[+] Дані успішно завантажені.


In [118]:
def select_clusters(df: pd.DataFrame, cluster_column: str, top_n: int = 2) -> pd.DataFrame:
    """
    Вибирає топ-N кластерів з найбільшою кількістю елементів.

    :param df: Вхідний DataFrame з кластерними мітками.
    :param cluster_column: Назва стовпця з кластерними мітками.
    :param top_n: Кількість найбільших кластерів для вибору.
    :return: DataFrame, що містить лише топ-N кластерів.
    """
    top_clusters = df[cluster_column].value_counts().nlargest(top_n).index.tolist()
    df_top = df[df[cluster_column].isin(top_clusters)].copy()
    print(f"Вибрано топ-{top_n} кластерів: {top_clusters}")
    return df_top

In [119]:
TOP_CLUSTERS = select_clusters(DATAFRAME, 'KMeans_Cluster')

Вибрано топ-2 кластерів: [0, 1]


In [120]:
def encode_clusters(df: pd.DataFrame, cluster_column: str) -> pd.DataFrame:
    """
    Кодує кластерні мітки у бінарні класи.

    :param df: Вхідний DataFrame з кластерними мітками.
    :param cluster_column: Назва стовпця з кластерними мітками.
    :return: DataFrame з доданим стовпцем 'Class'.
    """
    clusters = df[cluster_column].unique()
    if len(clusters) != 2:
        raise ValueError("[!] Кількість кластерів для бінарної класифікації має бути рівною 2.")

    df['Class'] = df[cluster_column].apply(lambda x: 1 if x == clusters[0] else 0)
    print("[+] Кластерні мітки успішно закодовані у бінарні класи.")
    return df

In [121]:
TOP_CLUSTERS = encode_clusters(TOP_CLUSTERS, 'KMeans_Cluster')

[+] Кластерні мітки успішно закодовані у бінарні класи.


In [122]:
def scale_data(df: pd.DataFrame, feature_columns: list[str]) -> tuple[np.ndarray, StandardScaler]:
    """
    Масштабує дані за допомогою стандартизації.

    :param df: Вхідний DataFrame.
    :param feature_columns: Список стовпців, що використовуються як ознаки.
    :return: Кортеж (масштабовані дані, об'єкт скейлера).
    """
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df[feature_columns])
    print("Дані успішно масштабується.")
    return X_scaled, scaler

In [123]:
feature_columns: list[str] = ['Value', 'Port Code']
X_scaled, scaler = scale_data(TOP_CLUSTERS, feature_columns)

Дані успішно масштабується.


In [124]:
def split_data(X: np.ndarray, y: np.ndarray, test_size: float = 0.3, random_state: int = 42) -> tuple[
    np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Розділяє дані на навчальну та тестову вибірки.

    :param X: Матриця ознак.
    :param y: Вектор цільових значень.
    :param test_size: Частка даних для тестової вибірки.
    :param random_state: Випадковий стан для відтворюваності.
    :return: Кортеж (X_train, X_test, y_train, y_test).
    """
    return train_test_split(X, y, test_size=test_size, random_state=random_state)


In [125]:
y: np.ndarray = TOP_CLUSTERS['Class'].values

X_train: np.ndarray
X_test: np.ndarray
y_train: np.ndarray
y_test: np.ndarray
X_train, X_test, y_train, y_test = split_data(X_scaled, y)
print(f"Розмір навчальної вибірки: {X_train.shape}")
print(f"Розмір тестової вибірки: {X_test.shape}")

Розмір навчальної вибірки: (3544, 2)
Розмір тестової вибірки: (1519, 2)


In [126]:
def find_optimal_k(X_train: np.ndarray, y_train: np.ndarray, X_test: np.ndarray, y_test: np.ndarray, min_k: int = 1,
                   max_k: int = 20, target_accuracy: float = 0.85) -> int:
    """
    Визначає оптимальну кількість сусідів (k), щоб досягти заданої точності.

    :param X_train: Навчальна матриця ознак.
    :param y_train: Навчальний вектор цільових значень.
    :param X_test: Тестова матриця ознак.
    :param y_test: Тестовий вектор цільових значень.
    :param min_k: Мінімальне значення k.
    :param max_k: Максимальне значення k.
    :param target_accuracy: Цільова точність.
    :return: Оптимальне значення k.
    """
    for k in range(min_k, max_k + 1):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        print(f"k={k}: Точність = {acc:.2f}")
        if acc >= target_accuracy:
            print(f"Оптимальне k = {k} з точністю {acc:.2f}")
            return k
    print("Не вдалося досягти цільової точності з заданим діапазоном k.")
    return max_k

In [127]:
optimal_k: int = find_optimal_k(X_train, y_train, X_test, y_test)

k=1: Точність = 1.00
Оптимальне k = 1 з точністю 1.00


In [128]:
def train_knn(X_train: np.ndarray, y_train: np.ndarray, k: int) -> KNeighborsClassifier:
    """
    Тренує модель K-Найближчих Сусідів.

    :param X_train: Навчальна матриця ознак.
    :param y_train: Навчальний вектор цільових значень.
    :param k: Кількість сусідів.
    :return: Навчена модель KNN.
    """
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    print(f"KNN модель з k={k} успішно натренована.")
    return knn

In [129]:
knn_model: KNeighborsClassifier = train_knn(X_train, y_train, optimal_k)

KNN модель з k=1 успішно натренована.


In [130]:
def evaluate(model: KNeighborsClassifier, X_test: np.ndarray, y_test: np.ndarray) -> None:
    """
    Оцінює модель KNN на тестових даних та виводить точність та звіт класифікації.

    :param model: Навчена модель KNN.
    :param X_test: Тестова матриця ознак.
    :param y_test: Тестовий вектор цільових значень.
    """
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Точність класифікації на тестовій вибірці: {accuracy:.2f}")
    print("Звіт класифікації:")
    print(classification_report(y_test, y_pred))

In [131]:
evaluate(knn_model, X_test, y_test)

Точність класифікації на тестовій вибірці: 1.00
Звіт класифікації:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       331
           1       1.00      1.00      1.00      1188

    accuracy                           1.00      1519
   macro avg       1.00      1.00      1.00      1519
weighted avg       1.00      1.00      1.00      1519


In [132]:
def cross_validation(model: KNeighborsClassifier, X: np.ndarray, y: np.ndarray, cv: int = 3) -> None:
    """
    Виконує крос-валідацію моделі KNN.

    :param model: Модель KNN.
    :param X: Матриця ознак.
    :param y: Вектор цільових значень.
    :param cv: Кількість блоків для крос-валідації.
    """
    scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
    print(f"Крос-валідація з {cv} блоками:")
    for fold, score in enumerate(scores, 1):
        print(f"Блок {fold}: Точність = {score:.2f}")
    print(f"Середня точність: {scores.mean():.2f}")
    print(f"Стандартне відхилення точності: {scores.std():.2f}")

In [133]:
cross_validation(knn_model, X_scaled, y, cv=3)

Крос-валідація з 3 блоками:
Блок 1: Точність = 1.00
Блок 2: Точність = 1.00
Блок 3: Точність = 1.00
Середня точність: 1.00
Стандартне відхилення точності: 0.00
